In [0]:
# %pip install requirements.txt

In [0]:
dbutils.library.restartPython()

In [0]:
import configparser
config = configparser.ConfigParser()
config.read('config/environment.ini')
config = config['CNC_MAINTENANCE_PREDICTOR']

In [0]:
config

In [0]:
def catalogs():
  return set(sql("SHOW CATALOGS").toPandas()['catalog'].to_list())

def schemas(catalog):
  return set(sql(f"SHOW DATABASES IN {catalog}").toPandas()['databaseName'].to_list())

In [0]:
if config['catalog'] not in catalogs():
    msg = f"Catalog [{config['catalog']}] does not exist"
    print(msg)
    try:
        _ = sql(f"CREATE CATALOG {config['catalog']}")
    except:
        raise Exception('Catalog creation failed')

# Set the catalog as default
sql(f"USE CATALOG {config['catalog']}")

# Check and create schema if it does not exist
if config['schema'] not in schemas(config['catalog']):
    msg = f"Schema [{config['catalog']}.{config['schema']}] does not exist"
    print(msg)
    try:
        _ = sql(f"CREATE DATABASE {config['catalog']}.{config['schema']}")
    except:
        raise Exception('Schema creation failed')

# Set the schema as default
sql(f"USE SCHEMA {config['schema']}")

In [0]:
bronze_table = f"{config['catalog']}.{config['schema']}.{config['bronze']}"
silver_table = f"{config['catalog']}.{config['schema']}.{config['silver']}"
gold_table = f"{config['catalog']}.{config['schema']}.{config['gold']}"
source_table = config['source']
experiment_name = config['experiment']
model_name = config['model']

In [0]:
import mlflow

username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
mlflow.set_experiment(f'/Users/{username}/{experiment_name}')